In [1]:
from pprint import pprint 
from elasticsearch import Elasticsearch 
es = Elasticsearch('http://localhost:9200',basic_auth=('elastic','Fe1odvmZ')) 
client_info = es.info() 
print(' Connected to Elasticsearch!') 
pprint(client_info.body) 

 Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'YkeBIv4WSkuNojcRv-HDJQ',
 'name': 'a15d97879705',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-06-18T22:09:56.772581489Z',
             'build_flavor': 'default',
             'build_hash': 'cc7302afc8499e83262ba2ceaa96451681f0609d',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '10.1.0',
             'minimum_index_compatibility_version': '8.0.0',
             'minimum_wire_compatibility_version': '8.18.0',
             'number': '9.0.3'}}


In [2]:
es.indices.delete(index='transcription',ignore_unavailable=True)
es.indices.create(
    index="transcription",
    settings={
        "index":{
            "number_of_shards":3,
            "number_of_replicas":2
        }
    },
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'transcription'})

In [ ]:
import json

ts_1445h = json.load(open("../transcriptions_1445h_v4/es_0T28iTJFhxw.json"))
ts_1445h

[{'video_link': '0T28iTJFhxw',
  'segment_index': 0,
  'start': 0.0,
  'end': 21.14,
  'text': 'بسم الله الرحمن الرحيم الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه ومن ولاه السلام عليكم جميعا ورحمة الله وبركاته وأهلا ومرحبا بكم في حلقة أخرى من هذا البرنامج وهذه الحلقات التي خصصناها للغة الإشارة لهذه الفئة الراقية كما تحدثنا بالأمس'},
 {'video_link': '0T28iTJFhxw',
  'segment_index': 1,
  'start': 21.14,
  'end': 36.4,
  'text': 'والتي تستخدم لغة الإشارة في التواصل بينها وتبحث من خلال ذلك في الأحكام الشرعية حتى تحدد موقعها في تعاملها مع المفردات التي يأمر الإسلام بضرورة أن يتقيد الإنسان فيها بالأحكام الشرعية'},
 {'video_link': '0T28iTJFhxw',
  'segment_index': 2,
  'start': 36.4,
  'end': 54.12,
  'text': 'وبحول الله تعالى سنناقش مجموعة من الأسئلة التي وردت إلينا عن طريقهم والتي سنستعرضها على الشاشة بإذن الله تعالى وهم مجموعة من مختلف أنحاء العالم طرحوا أسئلة جيدة ورائعة ستمثل إفادة لهم وإفادة لنا جميعا وللأخوة المتابعين'},
 {'video_link': '0T28iTJFhxw',
  'segment_index': 3,


In [4]:
def insert_document(document):
    response = es.index(index='transcription', body=document)
    return response


def print_info(response):
    print(f"""Document ID: {response['_id']} is '{
          response["result"]}' and is split into {response['_shards']['total']} shards.""")


for document in ts_1445h:
    response = insert_document(document)
    print_info(response)

Document ID: ITgTi5kBnwY8Mvf2uHLB is 'created' and is split into 3 shards.
Document ID: IjgTi5kBnwY8Mvf2uXIJ is 'created' and is split into 3 shards.
Document ID: IzgTi5kBnwY8Mvf2uXIT is 'created' and is split into 3 shards.
Document ID: JDgTi5kBnwY8Mvf2uXIb is 'created' and is split into 3 shards.
Document ID: JTgTi5kBnwY8Mvf2uXIm is 'created' and is split into 3 shards.
Document ID: JjgTi5kBnwY8Mvf2uXIw is 'created' and is split into 3 shards.
Document ID: JzgTi5kBnwY8Mvf2uXI5 is 'created' and is split into 3 shards.
Document ID: KDgTi5kBnwY8Mvf2uXJD is 'created' and is split into 3 shards.
Document ID: KTgTi5kBnwY8Mvf2uXJM is 'created' and is split into 3 shards.
Document ID: KjgTi5kBnwY8Mvf2uXJU is 'created' and is split into 3 shards.
Document ID: KzgTi5kBnwY8Mvf2uXJd is 'created' and is split into 3 shards.
Document ID: LDgTi5kBnwY8Mvf2uXJm is 'created' and is split into 3 shards.
Document ID: LTgTi5kBnwY8Mvf2uXJu is 'created' and is split into 3 shards.
Document ID: LjgTi5kBnwY8

In [6]:
import json
import os
from glob import glob
from tqdm import tqdm

# Function to process a single file
def process_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        documents = json.load(f)
    
    print(f"Inserting {len(documents)} documents from {os.path.basename(file_path)}")
    
    for document in tqdm(documents, desc="Inserting documents", leave=False):
        response = es.index(index='transcription', body=document)
        # Optional: store document IDs if needed
        # doc_ids.append(response['_id'])

# Process all transformed files
transformed_dir = "../transcriptions_1445h_v4"
json_files = glob(os.path.join(transformed_dir, "*.json"))

for file in tqdm(json_files, desc="Processing files"):
    process_file(file)

print("All documents inserted successfully!")

Processing files:   0%|          | 0/49 [00:00<?, ?it/s]

Inserting 153 documents from es_0T28iTJFhxw.json


Processing files:   2%|▏         | 1/49 [00:01<00:50,  1.04s/it]

Inserting 154 documents from es_18PJUffzGwA.json


Processing files:   4%|▍         | 2/49 [00:01<00:45,  1.04it/s]

...

Inserting 151 documents from es_YaeBiEXZa54.json


Processing files:  98%|█████████▊| 48/49 [00:42<00:00,  1.03it/s]

Inserting 141 documents from es_ZI89ktV79gU.json


Processing files: 100%|██████████| 49/49 [00:43<00:00,  1.12it/s]

All documents inserted successfully!


In [4]:
import json
import os
from glob import glob
from tqdm import tqdm
def insert_docs(doc):
    response=es.index(index='transcription',body=doc)
    return response

def print_info(res):
    print(f"""
Document Id: {res['_id']} is {res['result']} and is split into {res['_shards']['total']} shreds
          """)
    
docs_ids=[]

def process_transcription_file(file_path):
    """Process a single transcription file and insert all segments"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # print(f"Processing file: {os.path.basename(file_path)}")
        
        # Insert each segment as a separate document
        for i, segment in enumerate(tqdm(data['segment'],desc="Segments", leave=False)):
            doc = {
                'video_link': data['video_link'],
                'segment_index': i,
                'start': segment['start'],
                'end': segment['end'],
                'text': segment['text']
            }
            
            res = insert_docs(doc)
            docs_ids.append(res['_id'])
        
        
    except Exception as e:
        print(f"❌ Error processing {file_path}: {e}\n")

# Method 1: Process all JSON files in a directory
transcription_dir = "../transcriptions_1437"
json_files = glob(os.path.join(transcription_dir, "*.json"))

print(f"Found {len(json_files)} JSON files to process\n")

for file_path in json_files:
    process_transcription_file(file_path)

print("🎉 All files processed!")

Found 49 JSON files to process



🎉 All files processed!


In [2]:
#this is to show the datatypes of 
from pprint import pprint
index_mapping=es.indices.get_mapping(index='transcription')
pprint(index_mapping["transcription"]["mappings"]["properties"])

{'end': {'type': 'float'},
 'segment_index': {'type': 'long'},
 'start': {'type': 'float'},
 'text': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}},
          'type': 'text'},
 'video_link': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}},
                'type': 'text'}}


In [7]:
# count documents
es.count(index='transcription')

ObjectApiResponse({'count': 7403, '_shards': {'total': 3, 'successful': 3, 'skipped': 0, 'failed': 0}})

In [ ]:
#get a document
es.get(index='transcription',id=docs_ids[0])

ObjectApiResponse({'_index': 'transcription', '_id': '3tjeZ5kB4dmQKG5PqRxM', '_version': 1, '_seq_no': 2341, '_primary_term': 1, 'found': True, '_source': {'video_link': '0T28iTJFhxw', 'segment_index': 0, 'start': 0.0, 'end': 21.14, 'text': 'بسم الله الرحمن الرحيم الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه ومن ولاه السلام عليكم جميعا ورحمة الله وبركاته وأهلا ومرحبا بكم في حلقة أخرى من هذا البرنامج وهذه الحلقات التي خصصناها للغة الإشارة لهذه الفئة الراقية كما تحدثنا بالأمس'}})

In [3]:
# get doucment with query
query={
    "match":{
        "text":"مرض"
    }
}

es.count(index='transcription',query=query)

ObjectApiResponse({'count': 16, '_shards': {'total': 3, 'successful': 3, 'skipped': 0, 'failed': 0}})

In [4]:
# serch doucment with match word
query={
    "match":{
        "text":"صلاة السفر"
    }
}

# Execute search and get results
response = es.search(index='transcription', query=query, size=10)
# Print results
for hit in response['hits']['hits']:
    print(f"Video: {hit['_source']['video_link']}")
    print(f"Time: {hit['_source']['start']} - {hit['_source']['end']}")
    print(f"Text: {hit['_source']['text']}")
    print("-" * 50)

Video: pSrHxXY44AI
Time: 499.24 - 514.48
Text: سؤالين دكتور شخص تحرك من وطني بعد أذان الظهر ولم يكن ينوي السفر وقطع مسافة السفر فأخر صلاة الظهر مع العصر أي جمع تأخير
--------------------------------------------------
Video: WRTjOg8-B18
Time: 2381.58 - 2399.08
Text: فالصاحب السفر إنما هو لأجل السفر نعم طال هذا السفر أو قصر طالت يعني مدة المكث أو لم تطل نعم فهو اقتران و معية في أمر السفر نعم بهذا نفهم أن الله تبارك و تعالى نفى عن نفسه الصاحبة
--------------------------------------------------
Video: eUS0e-7tSHA
Time: 1857.16 - 1874.18
Text: لم نصلي التراويح لأننا سنصليها عند الرجوع و رجعنا في نفس اليوم و نسيت أن أصلي التراويح فماذا علي و ما حكم صلاة التراويح عند السفر نعم لا شيء عليهم و إن كان قد فاتهم فضل كبير
--------------------------------------------------
Video: x3jkF1y8hH4
Time: 1101.26 - 1120.26
Text: فأدنا وأقيمه ولي أمكما أفضلكما وهما اثنان فقط إذن فأجر الصلاة في جماعة في الفلاة حتى لا يدخل في النفوس وحشة من ترك صلاة الجماعة في المساجد بسبب السفر أو أن يدفع ذلك المسافر إلى أن ي

In [6]:
# search document with match word
query={
    "match":{
        "text":"صلاة السفر"
    }
}

# Execute search and get results
response = es.search(index='transcription', query=query, size=10)

# Print results
for i, hit in enumerate(response['hits']['hits']):
    video_link = hit['_source']['video_link']
    start_time = hit['_source']['start']
    
    # Create YouTube link with timestamp
    if 'youtube.com/watch?v=' in video_link or 'youtu.be/' in video_link:
        # Extract video ID
        if 'youtube.com/watch?v=' in video_link:
            video_id = video_link.split('v=')[1].split('&')[0]
        else:  # youtu.be format
            video_id = video_link.split('youtu.be/')[-1].split('?')[0]
        
        # Create timestamped YouTube link
        youtube_link_with_time = f"https://www.youtube.com/watch?v={video_id}&t={int(start_time)}s"
    else:
        youtube_link_with_time = f"{video_link}&t={int(start_time)}s"
    
    print(f"Result {i+1}:")
    print(f"Video: {video_link}")
    print(f"YouTube Link with Time: {youtube_link_with_time}")
    print(f"Time: {hit['_source']['start']} - {hit['_source']['end']}")
    print(f"Text: {hit['_source']['text']}")
    print("-" * 50)

# If you want just the first result with timestamp:
if response['hits']['hits']:
    first_hit = response['hits']['hits'][0]
    video_link = first_hit['_source']['video_link']
    start_time = int(first_hit['_source']['start'])
    
    # Extract video ID and create timestamped link
    if 'youtube.com/watch?v=' in video_link:
        video_id = video_link.split('v=')[1].split('&')[0]
    elif 'youtu.be/' in video_link:
        video_id = video_link.split('youtu.be/')[-1].split('?')[0]
    else:
        video_id = video_link  # Assume it's already a video ID
    
    timestamped_link = f"https://www.youtube.com/watch?v={video_id}&t={start_time}s"
    
    print(f"\n🎯 First Result with Timestamp:")
    print(f"Link: {timestamped_link}")

Result 1:
Video: pSrHxXY44AI
YouTube Link with Time: pSrHxXY44AI&t=499s
Time: 499.24 - 514.48
Text: سؤالين دكتور شخص تحرك من وطني بعد أذان الظهر ولم يكن ينوي السفر وقطع مسافة السفر فأخر صلاة الظهر مع العصر أي جمع تأخير
--------------------------------------------------
Result 2:
Video: WRTjOg8-B18
YouTube Link with Time: WRTjOg8-B18&t=2381s
Time: 2381.58 - 2399.08
Text: فالصاحب السفر إنما هو لأجل السفر نعم طال هذا السفر أو قصر طالت يعني مدة المكث أو لم تطل نعم فهو اقتران و معية في أمر السفر نعم بهذا نفهم أن الله تبارك و تعالى نفى عن نفسه الصاحبة
--------------------------------------------------
Result 3:
Video: eUS0e-7tSHA
YouTube Link with Time: eUS0e-7tSHA&t=1857s
Time: 1857.16 - 1874.18
Text: لم نصلي التراويح لأننا سنصليها عند الرجوع و رجعنا في نفس اليوم و نسيت أن أصلي التراويح فماذا علي و ما حكم صلاة التراويح عند السفر نعم لا شيء عليهم و إن كان قد فاتهم فضل كبير
--------------------------------------------------
Result 4:
Video: x3jkF1y8hH4
YouTube Link with Time: x3jkF1y8hH4&t=11